In [1]:
import time
from collections import namedtuple
import numpy as np
import tensorflow as tf

In [3]:
file_path = '/Users/li/workshop/MyRepository/DeepNaturalLanguageProcessing/DeepNLP/language_model/anna/data/anna.txt'
with open(file_path, 'r') as f:
    text = f.read()

vocab = set(text)
# print(vocab)
# 字符数字映射
vocab_to_int = {c: i for i, c in enumerate(vocab)}
# print(vocab_to_int)
# 数字字符映射
int_to_vocab = dict(enumerate(vocab))
# print(int_to_vocab)

# 对文本进行编码
encode = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
print(text[:100])
print('encode\n', encode[:100])

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everythin
encode
 [73 70 80 38 33 79  3 26 58  8  8  8 65 80 38 38 43 26 47 80 21 48 55 48 79
 59 26 80  3 79 26 80 55 55 26 80 55 48  0 79 53 26 79 23 79  3 43 26 13 17
 70 80 38 38 43 26 47 80 21 48 55 43 26 48 59 26 13 17 70 80 38 38 43 26 48
 17 26 48 33 59 26 71 24 17  8 24 80 43 54  8  8 74 23 79  3 43 33 70 48 17]


In [5]:
def generate_bath(arr, batch_size, seq_length):
    num_steps = batch_size * seq_length
    # print('num_steps', num_steps)
    n_iters = int(len(arr) / num_steps)
    # print('num_iters', n_iters)
    arr = arr[: num_steps * n_iters]
    # print('arr_b', arr[:,:15])
    # 重塑
    arr = arr.reshape((batch_size, -1))
    print('arr_a\n',arr[:,:15])
    # print(arr.shape[1])

    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n + seq_length]
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

batches = generate_bath(encode, 10, 50)
x, y = next(batches)
print('x.shape', np.shape(x))
print('x', x[:10, :10])
print('y.shape', np.shape(y))
print('y', y[:10, :10])

arr_a
 [[61 65 40 67 23 72 75 44 48  6  6  6  1 40 67]
 [44 40 37 44 54 31 23 44 58 31 50 54 58 44 23]
 [69 50 54 66  6  6 71  0 72 68 80 44 50 23  5]
 [54 44 62 16 75 50 54 58 44 65 50 68 44 49 31]
 [44 50 23 44 50 68 80 44 68 50 75 19 71 44 68]
 [44 73 23 44 32 40 68  6 31 54  3 36 44 32 65]
 [65 72 54 44 49 31 37 72 44 46 31 75 44 37 72]
 [ 7 44 79 16 23 44 54 31 32 44 68 65 72 44 32]
 [23 44 50 68 54  5 23 66 44 22 65 72 36  5 75]
 [44 68 40 50 62 44 23 31 44 65 72 75 68 72  3]]
x.shape (10, 50)
x [[61 65 40 67 23 72 75 44 48  6]
 [44 40 37 44 54 31 23 44 58 31]
 [69 50 54 66  6  6 71  0 72 68]
 [54 44 62 16 75 50 54 58 44 65]
 [44 50 23 44 50 68 80 44 68 50]
 [44 73 23 44 32 40 68  6 31 54]
 [65 72 54 44 49 31 37 72 44 46]
 [ 7 44 79 16 23 44 54 31 32 44]
 [23 44 50 68 54  5 23 66 44 22]
 [44 68 40 50 62 44 23 31 44 65]]
y.shape (10, 50)
y [[65 40 67 23 72 75 44 48  6  6]
 [40 37 44 54 31 23 44 58 31 50]
 [50 54 66  6  6 71  0 72 68 80]
 [44 62 16 75 50 54 58 44 65 50]
 [50 23 44 

In [6]:
def build_inputs(num_seqs, num_steps):
    '''
    构建输入层

    num_seqs: 每个batch中的序列个数
    num_steps: 每个序列包含的字符数
    '''
    inputs = tf.placeholder(tf.int32, shape=(num_seqs, num_steps), name='inputs')
    targets = tf.placeholder(tf.int32, shape=(num_seqs, num_steps), name='targets')

    # 加入keep_prob
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return inputs, targets, keep_prob

In [7]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    '''
    构建lstm层

    keep_prob
    lstm_size: lstm隐层中结点数目
    num_layers: lstm的隐层数目
    batch_size: batch_size

    '''
    # 构建一个基本lstm单元
    cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)

    # 添加dropout
    # drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

    # 堆叠
    # cell = tf.contrib.rnn.MultiRNNCell([drop for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)

    return cell, initial_state

In [8]:
def build_output(lstm_output, in_size, out_size):
    '''
    构造输出层

    lstm_output: lstm层的输出结果
    in_size: lstm输出层重塑后的size
    out_size: softmax层的size

    '''

    # 将lstm的输出按照列concate，例如[[1,2,3],[7,8,9]],
    # tf.concat的结果是[1,2,3,7,8,9]
    seq_output = tf.concat(lstm_output, axis=1)  # tf.concat(concat_dim, values)
    # reshape
    x = tf.reshape(seq_output, [-1, in_size])

    # 将lstm层与softmax层全连接
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))

    # 计算logits
    logits = tf.matmul(x, softmax_w) + softmax_b

    # softmax层返回概率分布
    out = tf.nn.softmax(logits, name='predictions')

    return out, logits

In [9]:
def build_loss(logits, targets, lstm_size, num_classes):
    '''
    根据logits和targets计算损失

    logits: 全连接层的输出结果（不经过softmax）
    targets: targets
    lstm_size
    num_classes: vocab_size

    '''

    # One-hot编码
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())

    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)

    return loss

In [10]:
def build_optimizer(loss, learning_rate, grad_clip):
    '''
    构造Optimizer

    loss: 损失
    learning_rate: 学习率

    '''

    # 使用clipping gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))

    return optimizer

In [11]:
class CharRNN:
    def __init__(self, num_classes, batch_size=64, num_steps=50,
                 lstm_size=128, num_layers=2, learning_rate=0.001,
                 grad_clip=5, sampling=False):

        # 如果sampling是True，则采用SGD
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()

        # 输入层
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # LSTM层
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        # 对输入进行one-hot编码
        x_one_hot = tf.one_hot(self.inputs, num_classes)

        # 运行RNN
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state

        # 预测结果
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)

        # Loss 和 optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [12]:
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

epochs = 20
# 每n轮进行一次变量保存
save_every_n = 200

In [13]:
model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers,
                learning_rate=learning_rate)

In [14]:
saver = tf.train.Saver(max_to_keep=100)

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in generate_bath(encode, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss,
                                                 model.final_state,
                                                 model.optimizer],
                                                feed_dict=feed)

            end = time.time()
            # control the print lines
            if counter % 100 == 0:
                print('轮数: {}/{}... '.format(e + 1, epochs),
                      '训练步数: {}... '.format(counter),
                      '训练误差: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end - start)))

            if (counter % save_every_n == 0):
                saver.save(sess, "/Users/li/workshop/MyRepository/DeepNaturalLanguageProcessing/DeepNLP/anna/checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

    saver.save(sess, "/Users/li/workshop/MyRepository/DeepNaturalLanguageProcessing/DeepNLP/anna/checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

arr_a
 [[61 65 40 ...,  1 40 67]
 [40 54 62 ..., 44 23 32]
 [69 50 23 ..., 31 23  6]
 ..., 
 [79 72 72 ...,  6 40  3]
 [40  3 32 ..., 72 44 67]
 [62 44 65 ..., 44 65 50]]


轮数: 1/20...  训练步数: 100...  训练误差: 2.6926...  3.4955 sec/batch


arr_a
 [[61 65 40 ...,  1 40 67]
 [40 54 62 ..., 44 23 32]
 [69 50 23 ..., 31 23  6]
 ..., 
 [79 72 72 ...,  6 40  3]
 [40  3 32 ..., 72 44 67]
 [62 44 65 ..., 44 65 50]]


轮数: 2/20...  训练步数: 200...  训练误差: 2.2999...  3.4753 sec/batch


轮数: 2/20...  训练步数: 300...  训练误差: 2.1590...  3.5773 sec/batch


...

arr_a
 [[61 65 40 ...,  1 40 67]
 [40 54 62 ..., 44 23 32]
 [69 50 23 ..., 31 23  6]
 ..., 
 [79 72 72 ...,  6 40  3]
 [40  3 32 ..., 72 44 67]
 [62 44 65 ..., 44 65 50]]


轮数: 8/20...  训练步数: 1400...  训练误差: 1.5696...  3.5623 sec/batch


轮数: 8/20...  训练步数: 1500...  训练误差: 1.4842...  3.4927 sec/batch


arr_a
 [[61 65 40 ...,  1 40 67]
 [40 54 62 ..., 44 23 32]
 [69 50 23 ..., 31 23  6]
 ..., 
 [79 72 72 ...,  6 40  3]
 [40  3 32 ..., 72 44 67]
 [62 44 65 ..., 44 65 50]]


轮数: 9/20...  训练步数: 1600...  训练误差: 1.4696...  3.5549 sec/batch


In [ ]:
def pick_top_n(preds, vocab_size, top_n=5):
    """
    从预测结果中选取前top_n个最可能的字符

    preds: 预测结果
    vocab_size
    top_n
    """
    p = np.squeeze(preds)
    # 将除了top_n个预测值的位置都置为0
    p[np.argsort(p)[:-top_n]] = 0
    # 归一化概率
    p = p / np.sum(p)
    # 随机选取一个字符
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [ ]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    """
    生成新文本

    checkpoint: 某一轮迭代的参数文件
    n_sample: 新闻本的字符长度
    lstm_size: 隐层结点数
    vocab_size
    prime: 起始文本
    """
    # 将输入的单词转换为单个字符组成的list
    samples = [c for c in prime]
    # sampling=True意味着batch的size=1 x 1
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # 加载模型参数，恢复训练
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            # 输入单个字符
            x[0, 0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state],
                                        feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        # 添加字符到samples中
        samples.append(int_to_vocab[c])

        # 不断生成字符，直到达到指定数目
        for i in range(n_samples):
            x[0, 0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state],
                                        feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])

    return ''.join(samples)

In [ ]:
tf.train.latest_checkpoint('checkpoints')

In [ ]:
# 选用最终的训练参数作为输入进行文本生成
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="The")
print(samp)